In [26]:
# !pip install -U pip uv
# !pip install -e git+https://github.com/BatsResearch/bonito#egg=bonito
!uv pip install datasets huggingface_hub spacy langchain pip install beautifulsoup4 requests unstructured zenml

Resolved 141 packages in 21.03s                                              
Downloaded 35 packages in 2.33s                                     
Installed 35 packages in 17ms                                       
 + alembic==1.8.1
 + azure-common==1.1.28
 + azure-core==1.30.1
 + azure-mgmt-core==1.4.0
 + azure-mgmt-resource==23.0.1
 + bcrypt==4.0.1
 - click==8.1.7
 + click==8.1.3
 + click-params==0.3.0
 - cloudpickle==3.0.0
 + cloudpickle==2.2.1
 + distro==1.9.0
 + docker==6.1.3
 + gitdb==4.0.11
 + gitpython==3.1.42
 + httplib2==0.19.1
 + ipywidgets==8.1.2
 + isodate==0.6.1
 + jupyterlab-widgets==3.0.10
 + mako==1.3.2
 + markdown-it-py==3.0.0
 + mdurl==0.1.2
 + passlib==1.7.4
 - pydantic==2.6.4
 + pydantic==1.10.14
 + pymysql==1.0.3
 + pyparsing==2.4.7
 + rich==13.7.1
 + smmap==5.0.1
 - sqlalchemy==2.0.28
 + sqlalchemy==1.4.41
 + sqlalchemy-utils==0.38.3
 + sqlalchemy2-stubs==0.0.2a38
 + sqlmodel==0.0.8
 - unstructured==0.12.6
 + unstructured==0.12.5
 + validators==0.18.2
 + websock

We can start by downloading all our documentation from our Gitbook docs located
at docs.zenml.io. We'll use the langchain scraper to make this easy.

In [16]:
from logging import getLogger
from typing import List, Set, Tuple
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

logger = getLogger(__name__)


def is_valid_url(url: str, base: str) -> bool:
    """
    Check if the given URL is valid and has the same base as the provided base.

    Args:
        url (str): The URL to check.
        base (str): The base URL to compare against.

    Returns:
        bool: True if the URL is valid and has the same base, False otherwise.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and parsed.netloc == base


def get_all_links(url: str, base: str) -> List[str]:
    """
    Retrieve all valid links from a given URL with the same base.

    Args:
        url (str): The URL to retrieve links from.
        base (str): The base URL to compare against.

    Returns:
        List[str]: A list of valid links with the same base.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    links = []

    for link in soup.find_all("a", href=True):
        href = link["href"]
        full_url = urljoin(url, href)
        parsed_url = urlparse(full_url)
        cleaned_url = parsed_url._replace(fragment="").geturl()
        if is_valid_url(cleaned_url, base):
            links.append(cleaned_url)

    return links


def crawl(url: str, base: str, visited: Set[str] = None) -> Set[str]:
    """
    Recursively crawl a URL and its links, retrieving all valid links with the same base.

    Args:
        url (str): The URL to crawl.
        base (str): The base URL to compare against.
        visited (Set[str]): A set of URLs that have been visited. Defaults to None.

    Returns:
        Set[str]: A set of all valid links with the same base.
    """
    if visited is None:
        visited = set()

    visited.add(url)
    links = get_all_links(url, base)

    for link in links:
        if link not in visited:
            visited.update(crawl(link, base, visited))

    return visited


def get_all_pages(url: str) -> List[str]:
    """
    Retrieve all pages with the same base as the given URL.

    Args:
        url (str): The URL to retrieve pages from.

    Returns:
        List[str]: A list of all pages with the same base.
    """
    logger.debug(f"Scraping all pages from {url}...")
    base_url = urlparse(url).netloc
    pages = crawl(url, base_url)
    logger.debug(f"Found {len(pages)} pages.")
    logger.debug("Done scraping pages.")
    return list(pages)

In [18]:
from langchain_community.document_loaders import UnstructuredURLLoader

DOCS_URL = "https://docs.zenml.io/v/0.55.5"

docs_urls = get_all_pages(DOCS_URL)

loader = UnstructuredURLLoader(urls=docs_urls)
loaded_docs = loader.load()

ImportError: unstructured package not found, please install it with `pip install unstructured`

In [ ]:
print(f"Fetched {len(loaded_docs)} documents.")